# Definição do Algoritmo

Nesse notebook iremos utilizar as bases de treino que construímos com o auxílio do scrip.py `feature_engineering.py`, presente na pasta `src/feature_engineering/`, para testar diversos algoritmos e identificar qual o melhor a ser utilizado para o problema em questão. Iremos testar desde modelos mais simples como Regressão Logística até os mais complexos como LightGBM e a EBM.


In [28]:
import pandas as pd
import numpy as np 
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from scipy.stats import ks_2samp
from utils import *

from lightgbm import LGBMClassifier
from interpret.glassbox import ExplainableBoostingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")

FEATURES = ['faixa_VAR_6', 'faixa_VAR_9', 'faixa_VAR_19', 'faixa_VAR_57', 'faixa_VAR_25', 'faixa_VAR_32', 'faixa_VAR_40', 'faixa_VAR_60', 'grupo_VAR_20']
TARGET = 'y'
# from utils import *

In [27]:
treino = pd.read_parquet('../../data/interim/treino.parquet')
teste = pd.read_parquet('../../data/interim/teste.parquet')

In [25]:
# Vamos treinar diversos modelos para depois comparar a suas performances
results = []

classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'Explainable Boosting Machine': ExplainableBoostingClassifier(random_state=42)
}

for name, clf in classifiers.items():
    print(f"Treinando e avaliando {name}...")
    
    # Treinar o modelo
    model = clf.fit(treino[FEATURES],treino[TARGET])
    y_train = treino[TARGET].values.flatten()
    y_test = teste[TARGET].values.flatten()

    # Gerar predições treino
    y_pred_treino = model.predict(treino[FEATURES])
    y_pred_proba_treino = model.predict_proba(treino[FEATURES])[:, 1]

    # Gerar predições teste 
    y_pred_teste = model.predict(teste[FEATURES])
    y_pred_proba_teste = model.predict_proba(teste[FEATURES])[:, 1]
    
    # Calcular ROC-AUC
    roc_auc_treino = roc_auc_score(treino[TARGET], y_pred_proba_treino)
    roc_auc_teste = roc_auc_score(teste[TARGET], y_pred_proba_teste)
    
    # Calcular Acurácia
    accuracy_treino = accuracy_score(treino[TARGET], y_pred_treino)
    accuracy_teste = accuracy_score(teste[TARGET], y_pred_teste)
    
    # Calcular KS
    ks_treino = calculate_ks(y_train, y_pred_proba_treino)
    ks_teste = calculate_ks(y_test, y_pred_proba_teste)
    
    # Armazenar os resultados
    results.append({
        'Classifier': name,
        'ROC-AUC Treino': roc_auc_treino*100,
        'ROC-AUC Teste': roc_auc_teste*100,
        'KS Stat Treino': ks_treino*100,
        'KS Stat Teste': ks_teste*100,
        'Accuracy Treino': accuracy_treino*100,
        'Accuracy Teste': accuracy_teste*100
    })

df_results = pd.DataFrame(results)
df_results.style.format({
        "ROC-AUC Treino": "{:,.2f}",
        "ROC-AUC Teste": "{:,.2f}",
        "KS Stat Treino": "{:,.2f}",
        "KS Stat Teste": "{:,.2f}",
        "Accuracy Treino": "{:,.2f}",
        "Accuracy Teste": "{:,.2f}"
    })

Treinando e avaliando Logistic Regression...
Treinando e avaliando Decision Tree...
Treinando e avaliando Random Forest...
Treinando e avaliando Gradient Boosting...
Treinando e avaliando LightGBM...
[LightGBM] [Info] Number of positive: 2316, number of negative: 5895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000337 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 8211, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.282061 -> initscore=-0.934263
[LightGBM] [Info] Start training from score -0.934263
Treinando e avaliando Explainable Boosting Machine...


,Classifier,ROC-AUC Treino,ROC-AUC Teste,KS Stat Treino,KS Stat Teste,Accuracy Treino,Accuracy Teste
0,Logistic Regression,68.82,70.12,27.95,31.66,72.32,68.94
1,Decision Tree,78.03,63.93,40.82,22.19,74.63,67.67
2,Random Forest,77.86,64.94,40.71,22.96,74.63,67.63
3,Gradient Boosting,70.54,69.79,30.23,30.34,72.72,68.94
4,LightGBM,73.76,68.44,34.85,27.40,72.91,69.05
5,Explainable Boosting Machine,69.30,70.36,28.30,31.95,72.37,68.54


## Conclusão 
* É possível de utilizar diferentes tipos de modelo para o problema em questão, com a exceção de da Árvore de Decisão e a Random Forest, que apresentaram indícios fortes de Overfitting. 
* Iremos tunar os hiperparâmetros do Gradient Boosting, LightGBM e da EBM e posteriormente avaliar o melhor modelo. 